dependencies

In [14]:
import pandas as pd
import numpy as np
import seaborn as sns
#import neattext.functions as nfx
from sklearn.naive_bayes import MultinomialNB
# Transformers
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

In [11]:
from transformers import AutoTokenizer

# Load Distilbert Tokenizer 
#model_ckpt = "distilbert-base-uncased"
model_ckpt = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [12]:
# Encode Our Example Text
encoded_text = tokenizer("The movie was not good")
tokens = tokenizer.convert_ids_to_tokens(encoded_text.input_ids)
print(encoded_text,len(encoded_text.input_ids))
print(tokens)

{'input_ids': [101, 1996, 3185, 2001, 2025, 2204, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]} 7
['[CLS]', 'the', 'movie', 'was', 'not', 'good', '[SEP]']


In [17]:
data = pd.read_csv(r'.\emotion_dataset_2.csv')

In [18]:
# Step 2: Splitting the Data
train_texts, test_texts, train_labels, test_labels = train_test_split(
    data['Clean_Text'], data['Emotion'], test_size=0.2, random_state=42
)

In [28]:
train_texts

27845                       Yes  anniversary come  Cheers 
25699                                        think bloody 
24688    Peace state fear kind unknown John Buchan Fill...
32367    fat bitches complain weight TOLD EAT MCDONALDS...
13168    Heb een lolly gekregen met 12 lollys erin en 1...
                               ...                        
16850                 Damn Im going stuck american time Xx
6265                                haha ill tie guys post
11284    jump wagon Worse case scenario die Till #alive...
860                                           bby plate :(
15795                                  friends try hurt me
Name: Clean_Text, Length: 27833, dtype: object

In [25]:
encoded_text = tokenizer(train_texts[0][0])
tokens = tokenizer.convert_ids_to_tokens(encoded_text.input_ids)
print(encoded_text,len(encoded_text.input_ids))
print(tokens)

TypeError: 'float' object is not subscriptable

In [36]:
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split

In [56]:
# Load the dataset
df = pd.read_csv(r'.\emotion_dataset_2.csv')
df['Clean_Text'] = df['Clean_Text'].fillna('')
df = df.head(800)

# Tokenization
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Encode text data
encoded_data = tokenizer(df['Clean_Text'].tolist(), padding=True, truncation=True, return_tensors='tf')

In [57]:
encoded_data

{'input_ids': <tf.Tensor: shape=(800, 46), dtype=int32, numpy=
array([[  101,   102,     0, ...,     0,     0,     0],
       [  101, 10878,  2552, ...,     0,     0,     0],
       [  101,  2126,  2188, ...,     0,     0,     0],
       ...,
       [  101,  8837,  2627, ...,     0,     0,     0],
       [  101,  8741, 20934, ...,     0,     0,     0],
       [  101,  9826,  2360, ...,     0,     0,     0]])>, 'token_type_ids': <tf.Tensor: shape=(800, 46), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])>, 'attention_mask': <tf.Tensor: shape=(800, 46), dtype=int32, numpy=
array([[1, 1, 0, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])>}

In [58]:
# Convert TensorFlow tensors to numpy arrays
X_data = encoded_data['input_ids'].numpy()
y_data = df['Emotion'].values

In [59]:
y_data

array(['neutral', 'joy', 'sadness', 'joy', 'joy', 'fear', 'sadness',
       'surprise', 'surprise', 'surprise', 'anger', 'sadness', 'joy',
       'fear', 'anger', 'sadness', 'surprise', 'joy', 'joy', 'surprise',
       'sadness', 'joy', 'fear', 'anger', 'fear', 'sadness', 'surprise',
       'joy', 'sadness', 'joy', 'joy', 'fear', 'fear', 'shame', 'fear',
       'joy', 'sadness', 'surprise', 'sadness', 'surprise', 'surprise',
       'anger', 'fear', 'sadness', 'anger', 'fear', 'sadness', 'joy',
       'anger', 'joy', 'joy', 'fear', 'sadness', 'sadness', 'sadness',
       'sadness', 'fear', 'fear', 'sadness', 'sadness', 'surprise',
       'neutral', 'joy', 'joy', 'surprise', 'fear', 'fear', 'fear',
       'neutral', 'fear', 'joy', 'joy', 'fear', 'joy', 'sadness', 'fear',
       'neutral', 'anger', 'fear', 'fear', 'fear', 'joy', 'anger',
       'anger', 'joy', 'surprise', 'surprise', 'surprise', 'anger',
       'surprise', 'disgust', 'anger', 'joy', 'fear', 'surprise',
       'sadness', '

In [60]:
# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42)

In [61]:
# Convert labels to one-hot encoding
#num_classes = len(df['Emotion'].unique())
#y_train_one_hot = tf.keras.utils.to_categorical(y_train, num_classes)
#y_test_one_hot = tf.keras.utils.to_categorical(y_test, num_classes)

from sklearn.preprocessing import LabelEncoder

# Encode labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

In [62]:
# Load pre-trained BERT model
bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_classes)

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = ['accuracy']
bert_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [63]:
# Train the model
history = bert_model.fit(X_train, y_train_encoded, validation_data=(X_test, y_test_encoded), epochs=3, batch_size=32)

Epoch 1/3
12/20 [=================>............] - ETA: 2:05 - loss: 1.8998 - accuracy: 0.2604

In [ ]:
# Evaluate the model
loss, accuracy = bert_model.evaluate(X_test, y_test_one_hot)
print(f'Loss: {loss}, Accuracy: {accuracy}')

# Save the model
bert_model.save_pretrained('bert_classification_model')